In [1]:
from PIL import Image
import keras
import glob
import cv2
import numpy as np
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

path_list = []
test=[]
width = 28
height = 28
for filename in glob.glob('C:/Users/user1/AI/Data/kaggle_dogcat/train/cats/*.jpg'): #assuming gif
    path_list.append(str(filename))

for filename in glob.glob('C:/Users/user1/AI/Data/kaggle_dogcat/train/dogs/*.jpg'): #assuming gif
    path_list.append(str(filename))
np.random.shuffle(path_list)
    
result_path = []
for filename in glob.glob('C:/Users/user1/AI\Data/kaggle_dogcat/test/*.jpg'): #assuming gif
    result_path.append(filename)
    

x_temp = np.array([cv2.resize(cv2.imread(this_path), (width,height)) for this_path in path_list])
y_temp=[]
for this_path in path_list:
    if "cats" in this_path:
        y_temp.append(1)
    else:
        y_temp.append(0)

x_train, x_test, y_train, y_test = train_test_split( x_temp, y_temp, test_size=.25, random_state=42)

x_predict = np.array([cv2.resize(cv2.imread(this_path), (width,height)) for this_path in result_path])

y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_predict = x_predict.astype('float32')

x_train /= 255
x_test /= 255
x_predict /= 255

Using TensorFlow backend.


In [2]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('x_predict shape:', x_predict.shape)
print(x_train.shape[0], 'train samples (x)')
print(x_test.shape[0], 'test samples (x)')
print(x_predict.shape[0], 'predict samples (x)')

print(y_train[0])
print('y_train shape:', y_train.shape)
print(y_train.shape[0], 'train samples (y)')
print(y_test.shape[0], 'test samples (y)')

x_train shape: (3000, 28, 28, 3)
x_test shape: (1000, 28, 28, 3)
x_predict shape: (400, 28, 28, 3)
3000 train samples (x)
1000 test samples (x)
400 predict samples (x)
[1. 0.]
y_train shape: (3000, 2)
3000 train samples (y)
1000 test samples (y)


In [3]:
"""
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        rescale=1./255)
gen.fit(x_train)
"""

'\nfrom keras.preprocessing.image import ImageDataGenerator\ngen = ImageDataGenerator(\n        rotation_range=20,\n        width_shift_range=0.2,\n        height_shift_range=0.2,\n        horizontal_flip=True,\n        rescale=1./255)\ngen.fit(x_train)\n'

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam

batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 2 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0818 16:52:25.985944 26032 deprecation_wrapper.py:119] From C:\Users\user1\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0818 16:52:26.003932 26032 deprecation_wrapper.py:119] From C:\Users\user1\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0818 16:52:26.008930 26032 deprecation_wrapper.py:119] From C:\Users\user1\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0818 16:52:26.040912 26032 deprecation_wrapper.py:119] From C:\Users\user1\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0818 16:52:26.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
__________

In [5]:
y_pred = model.predict(x_predict)[:,1]
# show the inputs and predicted outputs
for i in range(len(y_pred)):
	print("Predicted=%s" % (y_pred[i]))

TypeError: data type not understood